In [ ]:
import tensorflow as tf

# データロード
(x_train,y_train),(x_valid,y_valid) = tf.keras.datasets.cifar10.load_data()
x_train = x_train/255
x_valid = x_valid/255

# モデル定義
model = tf.keras.models.Sequential()
model.add(
    tf.keras.layers.Conv2D(
        filters=16,kernel_size=(3,3),padding="same",activation="relu",
        input_shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3])
    )
)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10,activation="softmax"))

# トレーニング
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.00001),
    metrics=['accuracy'],loss="sparse_categorical_crossentropy"
)
model.fit(x_train,y_train,batch_size=4,epochs=2,validation_data=(x_valid,y_valid))

# モデルの保存
model.save('./1')

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    entry_point='./src/4-1/train.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    entry_point='./src/4-2/train.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
model_uri = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
!aws s3 cp {model_uri} ./
!tar zxvf model.tar.gz

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('./1')
model.summary()